# Westeros Tutorial - Introducing emissions (part1/2): Adding emission bounds

In the first part, the model chose not to base the power system on wind power as electricity from wind turbines was more expensive than electricity produced from coal. However, we now introduce emissions to investigate the impact of climate policy. Let's see what happens then.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [1]:
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df

%matplotlib inline

<IPython.core.display.Javascript object>

C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\reporting\__init__.py:98: FutureWarning: Importing from genno.computations will be deprecated in a future version; use genno.operator instead.
  ("tom:nl-t-yv-ya", (genno.computations.add, "fom:nl-t-yv-ya", "vom:nl-t-yv-ya")),


In [2]:
mp = ixmp.Platform()

## Making a clone of the existing scenario 'baseline'

In [3]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "emission_bound",
    "introducing an upper bound on emissions",
    keep_solution=False,
)
scen.check_out()

In [4]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"

## Introducing Emissions

In [5]:
# First we introduce the emission of CO2 and the emission category GHG
scen.add_set("emission", "CO2")
scen.add_cat("emission", "GHG", "CO2")

# Then we add new units to the model library (needed only once)
mp.add_unit("tCO2/kWa")
mp.add_unit("MtCO2")

# Last we add CO2 emissions to the coal powerplant
emission_factor = make_df(
    "emission_factor",
    node_loc=country,
    year_vtg=vintage_years,
    year_act=act_years,
    mode="standard",
    unit="tCO2/kWa",
    technology="coal_ppl",
    emission="CO2",
    value=7.4,
)
scen.add_par("emission_factor", emission_factor)

# Add cumulative technology

In [6]:

scen.add_set("technology","coal_cumulative")
scen.add_set("relation","coal_cum")
years = sorted(set(act_years))

# create relation activity
list_relation = []
for ya in years:
    yr = [y for y in years if y <= ya]
    # print(yr)
    rel_act_coal_ppl = make_df("relation_activity",
                relation="coal_cum",
                node_rel=country,
                year_rel=ya,
                node_loc=country,
                technology="coal_ppl",
                year_act=yr,
                mode="standard",
                value=-1,
                unit = "GWa"
           )
    
    rel_act_coal_cumulative = make_df("relation_activity",
                relation="coal_cum",
                node_rel=country,
                year_rel=ya,
                node_loc=country,
                technology="coal_cumulative",
                year_act=ya,
                mode="standard",
                value=1,
                unit = "GWa"
           )
    list_relation += [rel_act_coal_ppl, rel_act_coal_cumulative]
df_relation = pd.concat(list_relation)

# create relation bounds
rel_eq = make_df("relation_upper",
        relation = "coal_cum",
        node_rel = country,
        year_rel = years,
        value = 0,
        unit = "GWa"
       )

# adding dataframe to parameters
scen.add_par("relation_activity", df_relation)
scen.add_par("relation_upper", rel_eq)
scen.add_par("relation_lower", rel_eq)

In [7]:
df_relation

,relation,node_rel,year_rel,node_loc,technology,year_act,mode,value,unit
0,coal_cum,Westeros,700,Westeros,coal_ppl,700,standard,-1,GWa
0,coal_cum,Westeros,700,Westeros,coal_cumulative,700,standard,1,GWa
0,coal_cum,Westeros,710,Westeros,coal_ppl,700,standard,-1,GWa
1,coal_cum,Westeros,710,Westeros,coal_ppl,710,standard,-1,GWa
0,coal_cum,Westeros,710,Westeros,coal_cumulative,710,standard,1,GWa
0,coal_cum,Westeros,720,Westeros,coal_ppl,700,standard,-1,GWa
1,coal_cum,Westeros,720,Westeros,coal_ppl,710,standard,-1,GWa
2,coal_cum,Westeros,720,Westeros,coal_ppl,720,standard,-1,GWa
0,coal_cum,Westeros,720,Westeros,coal_cumulative,720,standard,1,GWa


In [18]:
# adding share constraint of cumulative activity
scen.add_set("shares","coalstandard_share")

# creating lower and upper bound dataframes
mode_up = make_df("share_mode_up",
    shares = "coalstandard_share",
    node_share = country,
    technology = "coal_ppl",
    mode = "standard",
    year_act = 720,
    time = "year",
    value = 1,
    unit = "-"
)

mode_lo = make_df("share_mode_lo",
    shares = "coalstandard_share",
    node_share = country,
    technology = "coal_ppl",
    mode = "standard",
    year_act = 720,
    time = "year",
    value = 0,
    unit = "-"
)

# adding the parameters
scen.add_par("share_mode_up", mode_up)
scen.add_par("share_mode_lo", mode_lo)

## Define a Bound on Emissions

The `"type_year": "cumulative"` assigns an upper bound on the *weighted average of emissions* over the entire time horizon.

In [19]:
scen.add_par(
    "bound_emission", [country, "GHG", "all", "cumulative"], value=500.0, unit="MtCO2"
)

## Time to Solve the Model

In [20]:
scen.commit(comment="Introducing emissions and setting an upper bound")
scen.set_as_default()

In [21]:
scen.solve()

In [22]:
scen.var("OBJ")["lvl"]

206280.0625

## Checking Results

In [23]:
coal_ppl = scen.var("ACT", {"technology":"coal_ppl"})
coal_ppl

,node_loc,technology,year_vtg,year_act,mode,time,lvl,mrg
0,Westeros,coal_ppl,690,700,standard,year,18.264840,0.0
1,Westeros,coal_ppl,700,700,standard,year,29.109451,0.0
2,Westeros,coal_ppl,700,710,standard,year,29.109451,0.0
3,Westeros,coal_ppl,710,710,standard,year,47.716202,0.0
4,Westeros,coal_ppl,710,720,standard,year,47.716202,0.0
5,Westeros,coal_ppl,720,720,standard,year,30.786555,0.0


In [24]:
coal_ppl_cum = scen.var("ACT", {"technology":"coal_ppl"})
coal_ppl_cum["lvl"] = coal_ppl_cum["lvl"].cumsum()
coal_ppl_cum

,node_loc,technology,year_vtg,year_act,mode,time,lvl,mrg
0,Westeros,coal_ppl,690,700,standard,year,18.264840,0.0
1,Westeros,coal_ppl,700,700,standard,year,47.374292,0.0
2,Westeros,coal_ppl,700,710,standard,year,76.483743,0.0
3,Westeros,coal_ppl,710,710,standard,year,124.199945,0.0
4,Westeros,coal_ppl,710,720,standard,year,171.916147,0.0
5,Westeros,coal_ppl,720,720,standard,year,202.702703,0.0


In [25]:
coal_cumulative = scen.var("ACT", {"technology":"coal_cumulative"})
coal_cumulative

,node_loc,technology,year_vtg,year_act,mode,time,lvl,mrg
0,Westeros,coal_cumulative,700,700,standard,year,47.374292,0.0
1,Westeros,coal_cumulative,710,710,standard,year,124.199945,0.0
2,Westeros,coal_cumulative,720,720,standard,year,202.702703,0.0


In [26]:
rel_eq

,relation,node_rel,year_rel,value,unit
0,coal_cum,Westeros,700,0,GWa
1,coal_cum,Westeros,710,0,GWa
2,coal_cum,Westeros,720,0,GWa


## Close the connection to the database

In [27]:
mp.close_db()

In [31]:
trial = make_df("relation_activity",
                    relation="co2_storcum",
                    node_rel="country",
                    year_rel=2020,
                    node_loc="country",
                    technology="co2_stor",
                    year_act=[2020,2030],
                    mode="mode",
                    value=[-1,5],
                    unit = "-"
               )
trial

,relation,node_rel,year_rel,node_loc,technology,year_act,mode,value,unit
0,co2_storcum,country,2020,country,co2_stor,2020,mode,-1,-
1,co2_storcum,country,2020,country,co2_stor,2030,mode,5,-
